This post will be based in my Undergraduate thesis 'A Study on Gradient Boosting Algorithms and Hyperparameter Optimization using Optuna' 

- [Original GitHub repository](https://github.com/joaomh/study_boosting_optuna_USP_undergraduate_thesis)
- [Link to my Undergraduate thesis in PT-BR](https://bdta.abcd.usp.br/item/003122385)


# Tables of Content:

**1. [Introduction](#Introduction)**

**2. [Ensemble Learning](#Ensemble)**


**4. [AdaBoost](#Ada)** 

**5. [GBMs](#GBM)** 

**6. [XGBoost vs. CatBoost vs. LightGBM](#XGBoost)** 

**7. [Improving Metrics and Controlling Overfitting](#Additive)** 

**8. [Optuna in XGBoost vs. CatBoost vs. LightGBM](#Optuna)** 

**9. [Shap in XGBoost vs. CatBoost vs. LightGBM](#Shap)** 

**10. [Perfomance Comparison](#Additive)** 

**11. [Bibliography](#Bibliography)**


# Introduction

The purpose of this post is to introduce the fundamentals of boosting algorithms and the main difference between XGBoost, CatBoost and LightGBM. We will give the reader some necessary keys to well understand and use related methods and be able to design adapted solutions when needed.

If we look at the [2022 Kaggle Data Science & ML Survey](https://www.kaggle.com/kaggle-survey-2022), we can see that Gradient Boosting Machines (GBMs) have been widely used in recent years. They are supervised machine learning algorithms that have consistently produced excellent results across a wide range of problems and have won numerous machine learning competitions.


![png1](./img/kaggle_state.png)

# Ensemble 
Many machine learning models primarily aim for high prediction accuracy using a single model, where boosting algorithms strive to enhance predictions by sequentially training a series of weak models, with each model compensating for the weaknesses of its predecessors.

First of all we need to understand Ensemble Learning, it's based on the idea of combining several simpler prediction models (weak learner), training them for the same task, and producing from them a more complex grouped model (strong learner) that is the sum of its parts.

For example, when creating an ensemble model based on several decision trees, which are simple yet high-variance models (often considered 'weak learners'), we need to aggregate them to enhance their resistance to data variations. Therefore, it makes sense to train the trees separately, allowing each one to adapt to different parts of the dataset. This way, each tree gains knowledge about various data variations, collectively improving the ensemble's predictive performance.

There are various ensemble learning methods, but in this text, we will primarily focus on Boosting, which is used in GBMs, but we can mention three algorithms that aims at combining weak learners:

**Bagging**: It is generally done with homogeneous predictors, each one operating independently in relation to the others, in a parallel manner. The final algorithm is then constructed by aggregating the results obtained from the base models in some form of average. Random Forest is one of the most famous algorithm.

**Boosting**: Generally implemented with homogeneous predictors, applied sequentially where the posterior model depends on the predecessor, and then these models are combined in the final ensemble. GBMs work like this

**Stacking**: It is typically done with heterogeneous predictors, training them in parallel, and then combining their outputs by training a meta-model that generates predictions based on the predictions of the various weak models. Here we can for combine RandomForest with DecisionTree for example.

![png1](./img/boosting_bagging.png) [Image from Ensemble Learning: Bagging & Boosting](https://towardsdatascience.com/ensemble-learning-bagging-boosting-3098079e5422)

# AdaBoost
AdaBoost is a specific Boosting algorithm developed for classification problems hte original AdaBoost algorithm is designed for classification problems, where the output is either −1 or 1, and the final prediction for a given instance is a weighted sum of each generated weak classifier

$$
G(x) = sign\bigr[\sum^M_{m=1}\alpha_m\cdot G_m(x)\bigr]
$$
Here, the weights $\alpha_m$
are computed by the boosting algorithm, and the idea is to increase the influence of weak learners that are more accurate while simultaneously penalizing those that are not.
The weakness is identified by the weak estimator error rate
$$err_m = \frac{\sum_{i=1}^Nw_i\mathbf{I}(y_i\neq G_m(x_i))}{\sum_{i=1}^Nw_i}$$


1. Initialize the observation weights $w_i = 1/N, i = 1, 2, . . . , N .$
2. For $m=1$ to $M$:

    2.1. Fit a classifier $G_m(x)$ to the training data using weights $w_i$

    2.2. Compute $err_m = \frac{\sum_{i=1}^Nw_i\mathbf{1}(y_i\neq G_m(x_i))}{\sum_{i=1}^Nw_i}$

    2.3. Compute $\alpha_m = log((1-err_m)/err_m)$

    2.4. Set $w_i \rightarrow w_i\cdot exp[\alpha_m \cdot \mathbf{1}(y_i\neq G_m(x_i))],i=1,2,...,N$

3. Output $G(x) = sign\bigr[\sum^M_{m=1}\alpha_m\cdot G_m(x)\bigr]$

From [1][2]

![png](img/ada.png) [Marsh, Brendan (2016). Multivariate Analysis of the Vector Boson Fusion Higgs Boson](https://www.researchgate.net/publication/306054843_Multivariate_Analysis_of_the_Vector_Boson_Fusion_Higgs_Boson)

Scikit-Learn have a implementation of AdaBoost

In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X, y)
clf.predict([[0, 0, 0, 0]])
clf.score(X, y)

0.983

In [7]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
iris = load_iris()
X, y = iris.data, iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an AdaBoostClassifier with a base DecisionTreeClassifier
clf = AdaBoostClassifier(n_estimators=100, random_state=0)

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model
clf.score(X, y)

0.96

# GBMs
The Gradient Boosting Machines algorithm works by optimizing any given differentiable loss function, using gradient descent [3].

We can write de GBM model as 

$$F_M(x) = F_0(x) + \sum_{m=1}^MF_m(x)$$
$ \beta_mh(x; a_m)$ are the base functions learners, where $\beta_m$ is the weight, and $a_m$ the parameters of the learner $h$. And we have a loss function $L(y_i,F_m(x_i))$, so we would like to find all optimal values of this parameters that would minimize this loss funciton.
$$    \{\beta_m,\alpha_m\}_1^M = {\arg\min}_{\{\beta'_m,\alpha'_m\}_1^M}\sum_{i=1}^n L\Biggl(y^{(i)},\sum_{m=1}^M\beta'_mh(\mathbf{x}^{(i)};\alpha'_m)\Biggl)$$

In this situations where is infeasible we can try a 'greedy-stagewise' approach for $m=1,2,3,...,M$

$$(\beta_m,\alpha_m) = {\arg\min}_{\beta,\alpha}\sum_{i=1}^n L\Biggl(y^{(i)},F_{m-1}\mathbf{x}^{(i)} + \beta h(\mathbf{x}^{(i)};\alpha)\Biggl)$$
And then we can use a vectorized notation and make similar to the gradient descent formula. The learning rate, $\eta$ shrinks the influence of the new learner.
$$F_m(\mathbf{X}) = F_{m-1}(\mathbf{X}) + \eta \Delta_m(X)$$


The gradient of the loss function $L$ with relation to the last estimate $F_{m−1}(x)$ is,
$$-g_m(\mathbf{x}^{(i)}) = -\Bigg[\frac{\partial L(y^{(i)},c^{(i)})}{\partial F(\mathbf{x}^{(i)})}\Bigg]$$


Gradient of the loss function $L$ with respect to the last prediction is sometimes called pseudo-residual, and written as $r_{m−1}$ can be written as
$$\mathbf{r}_{m_1} = \nabla F_{m-1}(\mathbf{X})L(y,F_{m-1}(\mathbf{X})) = \nabla \hat{y}_{m-1}L(y,\hat{y}_{\mathbf{m-1}})$$


1. $F_0(\mathbf{X} = \arg\min_v\sum_{i=1}^n L(y^{(i)},v)$
2. For $m=1$ to $M$:

    2.1. $\mathbf{r}_{m_1} = \nabla \hat{y}_{m-1}L(y,\hat{y}_{\mathbf{m-1}})$ # Train a base learner minimizing squared error

    2.2. $\alpha = {\arg\min}_{\alpha,\beta}\sum_{i=1}^n(\mathbf{r}_{m-1}^{(i)}-\beta h(\mathbf{x}^{(i)};\alpha))^2$

    2.3. $\beta = {\arg\min}_{\beta}\sum_{i=1}^nL(y^{(i)},F_{m-1}(\mathbf{x}^{(i))}+\beta h(\mathbf{x}^{(i))};\alpha_m)$

    2.4. $\Delta_m(X) = \beta_mh(\mathbf{X};\alpha_m)$
    
    2.5 $F_m(\mathbf{X}) = F_{m-1}(\mathbf{X}) + \eta \Delta_m(X)$                                                              

3. Output $F_m$

From [3]


We also can find Gradient Boosting function in scikit-learn

In [11]:
# for classification
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(X,y)
model.score(X, y)

1.0

# XGBoost vs. CatBoost vs. LightGBM

# Bibliography
You can find all files in [this repository](https://github.com/joaomh/xgboost-catboost-lgbm)

Some references


[1]-[Schapire, Robert E(1999). A Short Introduction to Boosting](https://cseweb.ucsd.edu/~yfreund/papers/IntroToBoosting.pdf)

[2]-[HASTIE, T.; TIBSHIRANI, R.; FRIEDMAN, J. (2009). The Elements of Statistical Learning](https://hastie.su.domains/Papers/ESLII.pdf)

[3]-[Jerome H. Friedman (2001). GREEDY FUNCTION APPROXIMATION:
A GRADIENT BOOSTING MACHINE](https://jerryfriedman.su.domains/ftp/trebst.pdf)

- [Pinheiro, J., & Becker, M.. (2023). Um estudo sobre algoritmos de Boosting e a otimização de hiperparâmetros utilizando optuna.](https://bdta.abcd.usp.br/item/003122385)